# Inne podejście do przypisania sentymentu

* Uwzględnienie aby słowa nie były zbyt blisko innych kategorii.
* Aby odległość nie była zbyt duża
* Dalej pracujemy z wykluczoną kategorią neutralną

In [3]:
import re
import pandas as pd
import numpy as np
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
import gensim
from gensim import corpora, models
from timeit import default_timer as timer
from datetime import datetime, timedelta
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import spacy
from collections import Counter
import pyLDAvis
import pyLDAvis.gensim
from sklearn.model_selection import train_test_split
import scipy

importlib.reload(funcs)

<module 'funcs' from 'D:\\_projekty\\praca\\funcs.py'>

In [4]:
%time dane = pd.read_csv('00_dane/dane_cleanest.csv',  sep='|', encoding='UTF-8')
%time dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: re.sub(r'[\W\d]',' ', x))
%time dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: [word for word in x.split()])

Wall time: 14 s
Wall time: 32.7 s
Wall time: 5.67 s


## 1. Wczytanie listy słów z NAWL

Parametry wg których przypisano kategorie zmieniono aby zwiększyć liczbę przypisań. Usunięto grupę 'neutral'.

In [6]:
NAWL = pd.read_csv('00_dane/nawl-analysis_2.csv', encoding='UTF-8')
NAWL.drop('distance to N', axis=1, inplace=True)
NAWL.head()

,word,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D
0,tropiki,H,4.518518,1.703704,1.592593,2.074074,1.629630,3.637347,7.251989,7.312897,6.880706,7.236647
1,łączyć,H,3.814815,1.666667,1.925926,1.777778,1.666667,4.161421,6.728334,6.476909,6.615627,6.693552
2,bogaty,H,4.592593,2.370370,1.703704,1.703704,1.925926,3.826875,6.801012,7.372475,7.404916,7.217664
3,rozkoszny,H,5.296296,1.407407,1.481481,1.555556,1.370370,2.366185,7.513857,7.449380,7.387380,7.536883
4,nabytek,H,4.555555,1.666667,1.666667,1.555556,1.444444,3.227771,7.022504,6.998906,7.096976,7.198279


In [7]:
print('Lista zawiera {} słów.'.format(len(NAWL)))

Lista zawiera 2902 słów.


Analiza wczytanych danych

In [8]:
NAWL.groupby('category')["word"].count()

category
A     141
D      59
F     216
H    1193
S      93
U    1200
Name: word, dtype: int64

Klasa "U" oznacza brak klasyfikacji. Mamy więc mało przypisanych słów.

Tu zmiana - analiza jaka powinna być minimalna odległość do innych kategorii.

Inne:
- Happiness
- Anger
- Sadness
- Fear
- Neutral (zmia=eniono parametry tak, aby się nie dopsiywało)
- Disgust


In [12]:
NAWL[NAWL.category=='H'].describe()

,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D
count,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000
mean,4.232395,1.579655,1.605737,1.772719,1.406997,3.675458,7.030573,7.011613,6.864576,7.167292
std,0.905329,0.355472,0.375147,0.480650,0.289350,0.995655,0.439758,0.409107,0.410031,0.472033
min,2.038461,1.000000,1.000000,1.000000,1.000000,0.593512,6.216268,6.207362,6.201650,6.201617
25%,3.555556,1.307692,1.346154,1.423077,1.192308,3.013146,6.700125,6.712395,6.534533,6.812340
50%,4.230769,1.500000,1.518519,1.666667,1.346154,3.678711,6.999391,6.998906,6.832547,7.149659
75%,4.923077,1.769231,1.769231,2.038461,1.538462,4.352122,7.338378,7.275909,7.147334,7.496576
max,6.740741,3.538461,3.423077,4.074074,3.076923,6.177688,8.599053,8.338819,8.256472,9.016303


In [11]:
NAWL[NAWL.category=='A'].describe()

,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D
count,141.000000,141.000000,141.000000,141.000000,141.000000,141.000000,141.000000,141.000000,141.000000,141.000000
mean,1.600806,3.979683,2.767887,2.714544,2.707664,7.639365,5.562588,6.846685,6.889281,6.883089
std,0.370231,0.627824,0.591026,0.550433,0.598474,0.752416,0.431027,0.375129,0.475166,0.460734
min,1.038462,2.384615,1.423077,1.555556,1.423077,6.212498,3.906429,6.205494,6.202138,6.214353
25%,1.307692,3.576923,2.296296,2.296296,2.296296,7.093706,5.267837,6.537321,6.513632,6.520120
50%,1.555556,3.884615,2.703704,2.692308,2.629630,7.638972,5.607461,6.849047,6.830996,6.843368
75%,1.807692,4.423077,3.222222,3.076923,3.000000,8.247803,5.876029,7.084869,7.143684,7.179416
max,2.730769,5.230769,4.153846,4.076923,4.461538,9.113486,6.191297,8.005615,8.204575,8.187758


In [13]:
NAWL[NAWL.category=='S'].describe()

,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D
count,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000
mean,1.551022,2.885044,4.426125,3.324281,2.263977,7.960515,7.079855,5.464723,6.717270,7.570660
std,0.447637,0.517932,0.915800,0.820432,0.468261,0.935451,0.539734,0.527581,0.304584,0.695353
min,1.000000,1.846154,2.407408,1.884615,1.346154,6.203553,6.209384,4.037939,6.202651,6.269240
25%,1.192308,2.481482,3.814815,2.692308,1.961538,7.040774,6.607487,5.099032,6.528012,6.975377
50%,1.407407,2.814815,4.518518,3.259259,2.230769,8.007132,7.116718,5.578307,6.674991,7.571511
75%,1.884615,3.307692,5.148148,3.961539,2.518518,8.746274,7.445365,5.914396,6.882810,8.158342
max,3.230769,4.307692,6.346154,5.384615,3.555556,9.749147,8.523939,6.183913,7.510672,8.929523


In [14]:
NAWL[NAWL.category=='F'].describe()

,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D
count,216.000000,216.000000,216.000000,216.000000,216.000000,216.000000,216.000000,216.000000,216.000000,216.000000
mean,1.764351,2.692519,2.689788,3.980598,2.468404,7.450890,6.907823,6.917212,5.554597,7.090821
std,0.445285,0.632086,0.667011,0.757233,0.511415,0.848951,0.432678,0.401337,0.482439,0.535342
min,1.000000,1.555556,1.500000,2.038461,1.307692,6.218613,6.210329,6.216388,3.549725,6.214032
25%,1.444444,2.185185,2.192308,3.495370,2.115385,6.699456,6.611008,6.616360,5.305897,6.661308
50%,1.692308,2.653846,2.653846,3.962963,2.415242,7.356948,6.845508,6.887094,5.646307,7.094750
75%,2.009615,3.112180,3.085470,4.500000,2.814815,8.093633,7.134506,7.187774,5.927036,7.402247
max,3.074074,4.407407,5.076923,6.153846,4.000000,9.809220,8.562069,8.063036,6.198540,8.882463


In [15]:
NAWL[NAWL.category=='D'].describe()

,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D
count,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000
mean,1.487179,2.770197,2.440919,2.762519,4.362765,7.863393,7.048225,7.322565,7.058519,5.257185
std,0.363492,0.675602,0.569039,0.651330,0.853119,0.865375,0.469201,0.526915,0.501163,0.711925
min,1.037037,1.500000,1.269231,1.444444,2.692308,6.265155,6.288564,6.397032,6.308234,3.527269
25%,1.192308,2.288462,2.056268,2.250000,3.792735,7.224362,6.659409,6.848284,6.670975,4.852075
50%,1.423077,2.777778,2.384615,2.692308,4.269231,7.977880,7.042188,7.269420,7.023906,5.381371
75%,1.711539,3.188746,2.849003,3.282764,5.134615,8.552730,7.390201,7.749788,7.353300,5.778650
max,2.407408,4.576923,3.769231,4.307692,5.777778,9.538703,8.148120,8.246947,8.423235,6.169866


In [17]:
a = NAWL.iloc[100]
a

word              orchidea
category                 H
mean Happiness     4.22222
mean Anger         1.51852
mean Sadness       1.74074
mean Fear          1.48148
mean Disgust       1.59259
distance to H      3.69759
distance to A      7.15178
distance to S      6.83866
distance to F      7.17292
distance to D      7.07942
Name: 100, dtype: object

Przypiszmy mimo wszystko flagę, do której odległość jest najmniejsza.

Dodajemy dwa dodatkowe parametry, które pozwolą zminimalizować złe przypisania i zachować rozdzielność zbiorów.

In [37]:
NAWL['new_flag']=''
for i in range(len(NAWL)):
    a = NAWL.iloc[i]
    l = [a['distance to H'], a['distance to A'], a['distance to S'], a['distance to F'], a['distance to D']]
    l.sort()
    m=min(l)
    if (l[1]-m>0.5) & (m<6.2):
        if m == a['distance to H']:
            NAWL.loc[i, 'new_flag']='H'
        elif m == a['distance to S']:
            NAWL.loc[i, 'new_flag']='S'
        elif m == a['distance to F']:
            NAWL.loc[i, 'new_flag']='F'
        elif m == a['distance to D']:
            NAWL.loc[i, 'new_flag']='D'
        elif m == a['distance to A']:
            NAWL.loc[i, 'new_flag']='A'

In [38]:
NAWL.groupby('new_flag')["word"].count()

#new_flag
#A     322
#D     149
#F     465
#H    1740
#S     226
#Name: word, dtype: int64'''

new_flag
      829
A     149
D      67
F     226
H    1536
S      95
Name: word, dtype: int64

In [40]:
NAWL[(NAWL.new_flag=='') & (NAWL.category!='U')]

,word,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,new_flag
249,lekarz,H,3.148148,2.925926,2.629630,3.259259,2.222222,6.008564,6.831506,7.153910,6.464431,7.487717,
356,duchowny,H,2.461539,2.461539,2.076923,2.153846,2.346154,5.939712,6.329299,6.727291,6.667926,6.463308,
365,państwo,H,2.500000,2.346154,2.461539,2.346154,2.038461,5.948227,6.335016,6.283603,6.407092,6.703123,
438,odsetki,H,2.807692,2.846154,2.769231,3.269231,2.307692,6.177688,7.023087,7.077578,6.477410,7.448467,
440,centralny,H,2.038461,1.884615,1.961538,2.192308,2.230769,6.171588,6.631926,6.586398,6.385302,6.351066,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1640,szpiegostwo,F,1.888889,3.370370,2.481482,3.703704,3.185185,7.613279,6.504817,7.316831,6.111363,6.661264,
1651,niezdrowy,D,1.423077,2.961539,3.076923,3.000000,3.192308,7.635745,6.552183,6.397032,6.432811,6.164684,
1658,skacowany,D,1.814815,3.407407,3.296296,2.481482,3.740741,7.630841,6.597960,6.642764,7.388630,6.161181,
1673,wilgotny,D,2.230769,2.538461,1.923077,2.076923,3.038461,6.265155,6.288564,6.859811,6.682882,5.768455,


In [41]:
NAWL = NAWL[NAWL.new_flag!='']

In [42]:
NAWL.shape

(2073, 13)

## 2. Przypisanie sentymentu słów

### 2.1 Budowa słownika wszystkich słów

In [43]:
dict_sp = {}
for sp in dane['speech_rawest']:
    for w in sp:
        if w not in dict_sp.keys():
            dict_sp[w]=0
        dict_sp[w]+=1

In [44]:
print('Liczba unikalnych słów w słowniku wynosi {}.'.format(len(dict_sp)))

Liczba unikalnych słów w słowniku wynosi 152498.


In [45]:
df_sp = pd.DataFrame(list(dict_sp.items()), columns=['word', 'counter'])
df_sp.head()

,word,counter
0,chcieć,205353
1,zdanie,46410
2,powiedzieć,114946
3,temat,36539
4,skutek,31434


In [46]:
len(df_sp)

152498

### 2.2 Dogranie parametrów

In [47]:
df_sp = pd.merge(df_sp, NAWL, left_on='word', right_on='word', how='left' )

In [49]:
df_sp[300:310]

,word,counter,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,new_flag
300,zły,23333,A,1.333333,4.592593,3.370370,3.592593,2.518518,8.549291,5.734484,6.945962,6.831202,7.815662,A
301,doświadczenie,14008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302,przeszłość,4463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,formułować,2516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,absolutnie,5985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,związany,19795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,postulować,3699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,chodzić,104920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,nowy,86180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309,ocena,27663,F,2.884615,3.000000,2.730769,4.038462,1.923077,6.422515,6.701242,6.919094,5.542771,7.558140,F


Ile mamy dogranych?

In [50]:
len(df_sp[pd.isnull(df_sp['category'])==False])

1918

In [51]:
pickle.dump(df_sp, open('df_sp.p', 'wb'))

## 3. Dogranie embeddingów

In [9]:
df_sp = pickle.load(open('df_sp.p','rb'))

In [11]:
df_sp.head()

,word,counter,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,distance to N,new_flag
0,chcieć,205353,H,4.333334,1.777778,2.037037,2.148148,1.592593,4.030499,7.033235,6.847620,6.729651,7.201385,4.369833,H
1,zdanie,46410,H,2.962963,1.222222,1.185185,1.444444,1.148148,4.453825,6.560185,6.583048,6.347609,6.579155,2.303450,H
2,powiedzieć,114946,U,3.000000,1.346154,1.538462,2.307692,1.192308,5.129413,6.853925,6.679800,5.946053,6.967851,2.799579,H
3,temat,36539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,skutek,31434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



### 3.1 Pobranie modelu dla języka polskiego

In [52]:
import fasttext.util
#fasttext.util.download_model('pl', if_exists='ignore')  
ft = fasttext.load_model('cc.pl.300.bin')

### 3.2 Słowniki pomocnicze

In [53]:
NAWL_vec_dict = {}
for w in NAWL["word"]:
    NAWL_vec_dict[w] = ft.get_word_vector(w)
corp_vec_dict = {}
for w in df_sp["word"]:
    corp_vec_dict[w] = ft.get_word_vector(w)    

In [54]:
pickle.dump(NAWL_vec_dict, open('nawl.p', 'wb'))
pickle.dump(corp_vec_dict, open('corp_vec.p', 'wb'))
                        

### 3.3 Przypisanie sentymentu na podstawie pięciu najbliższych słów z listy NAWL

In [72]:
neigh_cnt =5 #parametr z ilu słów korzystamy
max_dist = 0.7

for word in df_sp['word'][pd.isnull(df_sp['new_flag'])==True]:

    word_vec = corp_vec_dict[word]
    word_dict = {}
    neigh_df = pd.DataFrame(columns=['distance to H','distance to A','distance to S','distance to F','distance to D'])
    
    #obliczamy odległość do każdego słowa z NAWL
    for k in NAWL_vec_dict.keys():
        word_dict[k] = scipy.spatial.distance.cosine(word_vec, NAWL_vec_dict[k])

    #dla 'neigh_cnt' najbliższym (pod warunkiem, że odległość nie jest bardzo duża)
    for neighbor, dist in sorted(word_dict.items(), key=lambda item: item[1])[:neigh_cnt]:
        if dist<=max_dist:
            neigh_df = neigh_df.append(NAWL[['distance to H','distance to A','distance to S','distance to F','distance to D']][NAWL['word']==neighbor] )

    n_stats = neigh_df.mean()
    if len(neigh_df>0):
        df_sp.loc[df_sp.word==word, 'distance to H'] = n_stats['distance to H']
        df_sp.loc[df_sp.word==word, 'distance to A'] = n_stats['distance to A']
        df_sp.loc[df_sp.word==word, 'distance to S'] = n_stats['distance to S']
        df_sp.loc[df_sp.word==word, 'distance to F'] = n_stats['distance to F']
        df_sp.loc[df_sp.word==word, 'distance to D'] = n_stats['distance to D']

       
            
        l = list(neigh_df.mean())
        l.sort()
        m=min(l)
        if (l[1]-m>0.5) & (m<6.2):
            if m == neigh_df.mean()['distance to H']:
                df_sp.loc[df_sp.word==word, 'new_flag']='H'
            elif m == neigh_df.mean()['distance to S']:
                df_sp.loc[df_sp.word==word, 'new_flag']='S'
            elif m == neigh_df.mean()['distance to F']:
                df_sp.loc[df_sp.word==word, 'new_flag']='F'
            elif m == neigh_df.mean()['distance to D']:
                df_sp.loc[df_sp.word==word, 'new_flag']='D'
            elif m == neigh_df.mean()['distance to A']:
                df_sp.loc[df_sp.word==word, 'new_flag']='A'

C:\Users\Magda\Anaconda3\lib\site-packages\scipy\spatial\distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [73]:
print("Uzupełniony sentyment dla {} z {} słów.".format(len(df_sp[pd.isnull(df_sp['new_flag'])==False]), len(df_sp)))

Uzupełniony sentyment dla 112567 z 152498 słów.


In [74]:
pickle.dump(df_sp, open('df_sp.p', 'wb'))

In [75]:
df_sp[df_sp.word=='skutek']

,word,counter,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,new_flag
4,skutek,31434,NaN,NaN,NaN,NaN,NaN,NaN,4.623716,6.783997,6.879018,6.202185,7.055279,H
